In [2]:
import tushare as ts
import pandas as pd
import numpy as np

## get raw data from tushare

In [3]:
pro=ts.pro_api("***") # tushare key
data=pro.query('stock_basic', exchange='', list_status='L', fields='ts_code,market,exchange,industry,list_date,list_status,delist_date')
ts_code_list=data["ts_code"].tolist()

df=None
for i,ts_code in enumerate(ts_code_list):
    new_df = pro.daily(**{
        "ts_code": ts_code,
        "trade_date": "",
        "start_date": 20100101,
        "end_date": 20201231,
        "offset": "",
        "limit": ""
    }, fields=[
        "ts_code",
        "trade_date",
        "open",
        "high",
        "low",
        "close",
        "pre_close",
        "change",
        "pct_chg",
        "vol",
        "amount"
    ])
    df=new_df if df is None else pd.concat([df,new_df])

In [4]:
df_daily=df.sort_values(by=['ts_code', 'trade_date'], ascending=[True, True])
df_daily=df_daily.reset_index()
df_daily.head()

In [6]:
# get return
df_daily["return"]=df_daily.groupby("ts_code").shift(-2).apply(lambda x:x["close"]/x["pre_close"]-1,axis=1)
df_daily[abs(df_daily["return"])>0.95]=np.nan

## FE: manual

In [11]:
## price
df_daily[["sma_open","sma_close","sma_high","sma_low"]]=\
df_daily.groupby("ts_code")[["open","close","high","low"]].rolling(5).mean().values

for col in ["open","close","high","low"]:
    df_daily[col+"_s0"]=df_daily.groupby("ts_code").shift(0)[col]*5/15
    df_daily[col+"_s1"]=df_daily.groupby("ts_code").shift(1)[col]*4/15
    df_daily[col+"_s2"]=df_daily.groupby("ts_code").shift(2)[col]*3/15
    df_daily[col+"_s3"]=df_daily.groupby("ts_code").shift(3)[col]*2/15
    df_daily[col+"_s4"]=df_daily.groupby("ts_code").shift(4)[col]*1/15
    df_daily["wma_"+col]=df_daily[[col+"_s0",col+"_s1",col+"_s2",col+"_s3",col+"_s4"]].sum(axis=1,skipna=False)
    
    df_daily[col+"_s1"]=df_daily.groupby("ts_code").shift(1)[col]
    df_daily["diff_"+col+"_s0"]=df_daily[col]/df_daily[col+"_s1"]-1
    df_daily["diff_"+col+"_s1"]=df_daily.groupby("ts_code").shift(1)["diff_"+col+"_s0"]
    df_daily["diff_"+col+"_s2"]=df_daily.groupby("ts_code").shift(2)["diff_"+col+"_s0"]
    df_daily["diff_"+col+"_s3"]=df_daily.groupby("ts_code").shift(3)["diff_"+col+"_s0"]
    df_daily["diff_"+col+"_s4"]=df_daily.groupby("ts_code").shift(4)["diff_"+col+"_s0"]
    df_daily["sma_diff_"+col]=df_daily[["diff_"+col+"_s0","diff_"+col+"_s1","diff_"+col+"_s2","diff_"+col+"_s3","diff_"+col+"_s4"]].mean(axis=1,skipna=False)
    
    df_daily["norm_sma_"+col]=df_daily[col]/df_daily["sma_"+col]-1
    df_daily["norm_wma_"+col]=df_daily[col]/df_daily["wma_"+col]-1
    
    df_daily=df_daily.drop([col+"_s0",col+"_s1",col+"_s2",col+"_s3",col+"_s4"],axis=1)

In [12]:
## change pct_chg
df_daily[["sma_change","sma_pct_chg"]]=\
df_daily.groupby("ts_code")[["change","pct_chg"]].rolling(5).mean().values

for col in ["change","pct_chg"]:
    df_daily[col+"_s0"]=df_daily.groupby("ts_code").shift(0)[col]*5/15
    df_daily[col+"_s1"]=df_daily.groupby("ts_code").shift(1)[col]*4/15
    df_daily[col+"_s2"]=df_daily.groupby("ts_code").shift(2)[col]*3/15
    df_daily[col+"_s3"]=df_daily.groupby("ts_code").shift(3)[col]*2/15
    df_daily[col+"_s4"]=df_daily.groupby("ts_code").shift(4)[col]*1/15
    df_daily["wma_"+col]=df_daily[[col+"_s0",col+"_s1",col+"_s2",col+"_s3",col+"_s4"]].sum(axis=1,skipna=False)
    df_daily=df_daily.drop([col+"_s0",col+"_s1",col+"_s2",col+"_s3",col+"_s4"],axis=1)

In [13]:
## vol, amount
df_daily["log_vol"]=np.log(df_daily["vol"])
df_daily["log_vol_s1"]=df_daily.groupby("ts_code").shift(1)["log_vol"]
df_daily["diff_log_vol"]=df_daily["log_vol"]/df_daily["log_vol_s1"]-1
df_daily=df_daily.drop(["log_vol_s1"],axis=1)
df_daily["sma_diff_log_vol"]=df_daily.groupby("ts_code")["diff_log_vol"].rolling(5).mean().values

df_daily["log_amount"]=np.log(df_daily["amount"])
df_daily["log_amount_s1"]=df_daily.groupby("ts_code").shift(1)["log_amount"]
df_daily["diff_log_amount"]=df_daily["log_amount"]/df_daily["log_amount_s1"]-1
df_daily=df_daily.drop(["log_amount_s1"],axis=1)
df_daily["sma_diff_log_amount"]=df_daily.groupby("ts_code")["diff_log_amount"].rolling(5).mean().values

In [16]:
## ctc
df_daily["ctc"]=df_daily["close"]/df_daily["pre_close"]-1

col="ctc"
df_daily[col+"_s0"]=df_daily.groupby("ts_code").shift(0)[col]*5/15
df_daily[col+"_s1"]=df_daily.groupby("ts_code").shift(1)[col]*4/15
df_daily[col+"_s2"]=df_daily.groupby("ts_code").shift(2)[col]*3/15
df_daily[col+"_s3"]=df_daily.groupby("ts_code").shift(3)[col]*2/15
df_daily[col+"_s4"]=df_daily.groupby("ts_code").shift(4)[col]*1/15
df_daily["wma_"+col]=df_daily[[col+"_s0",col+"_s1",col+"_s2",col+"_s3",col+"_s4"]].sum(axis=1,skipna=False)

col="ctc"
df_daily[col+"_s0"]=df_daily.groupby("ts_code").shift(0)[col]
df_daily[col+"_s1"]=df_daily.groupby("ts_code").shift(1)[col]
df_daily[col+"_s2"]=df_daily.groupby("ts_code").shift(2)[col]
df_daily[col+"_s3"]=df_daily.groupby("ts_code").shift(3)[col]
df_daily[col+"_s4"]=df_daily.groupby("ts_code").shift(4)[col]

df_daily[["sma_ctc"]]=\
df_daily.groupby("ts_code")[["ctc_s0"]].rolling(5).mean().values

df_daily=df_daily.drop(["ctc"],axis=1)

In [4]:
feature_list=[
    "change","sma_change","wma_change",
    "pct_chg","sma_pct_chg","wma_pct_chg",
    "diff_open_s0","diff_open_s1","diff_open_s2","diff_open_s3","diff_open_s4","sma_diff_open","norm_sma_open","norm_wma_open",
    "diff_close_s0","diff_close_s1","diff_close_s2","diff_close_s3","diff_close_s4","sma_diff_close","norm_sma_close","norm_wma_close",
    "diff_high_s0","diff_high_s1","diff_high_s2","diff_high_s3","diff_high_s4","sma_diff_high","norm_sma_high","norm_wma_high",
    "diff_low_s0","diff_low_s1","diff_low_s2","diff_low_s3","diff_low_s4","sma_diff_low","norm_sma_low","norm_wma_low",
    "diff_log_vol","sma_diff_log_vol","diff_log_amount","sma_diff_log_amount",
]
df_daily_clean=df_daily.dropna(axis=0)
df_daily_feature=df_daily_clean[feature_list].copy()

## feature normalization: clip

In [ ]:
## change pct_chg
for col in ["change","sma_change","wma_change","pct_chg","sma_pct_chg","wma_pct_chg"]:
    t=df_daily_feature[[col]].quantile([.05,.95]).abs().mean().values
    print(t)
    df_daily_feature.loc[:,col]=df_daily_feature[[col]].clip(-t,t)
    
## price
for col_part1 in ["open","close","high","low"]:
    for col_part2 in ["s0","s1","s2","s3","s4"]:
        col = "diff_"+col_part1+"_"+col_part2
        t=df_daily_feature[[col]].quantile([.01,.99]).abs().mean().values
        print(t)
        df_daily_feature.loc[:,col]=df_daily_feature[[col]].clip(-t,t)
    
    for col_part3 in ["sma_diff_","norm_sma_","norm_wma_"]: 
        col=col_part3+col_part1
        t=df_daily_feature[[col]].quantile([.01,.99]).abs().mean().values
        print(t)
        df_daily_feature.loc[:,col]=df_daily_feature[[col]].clip(-t,t)
        
## vol,amount
for col in ["diff_log_vol","sma_diff_log_vol","diff_log_amount","sma_diff_log_amount"]:
    t=df_daily_feature[[col]].quantile([.01,.99]).abs().mean().values
    print(t)
    df_daily_feature.loc[:,col]=df_daily_feature[[col]].clip(-t,t)

## feature normalization: zscore 

In [12]:
# except change & pct_chg
df_tmp=df_daily_feature[["change","sma_change","wma_change","pct_chg","sma_pct_chg","wma_pct_chg"]].copy()
df_daily_feature = (df_daily_feature - df_daily_feature.mean())/df_daily_feature.std(ddof=0)
df_daily_feature[["change","sma_change","wma_change","pct_chg","sma_pct_chg","wma_pct_chg"]]=df_tmp[["change","sma_change","wma_change","pct_chg","sma_pct_chg","wma_pct_chg"]]

In [13]:
df_daily_feature.describe()

,change,sma_change,wma_change,pct_chg,sma_pct_chg,wma_pct_chg,diff_open_s0,diff_open_s1,diff_open_s2,diff_open_s3,...,diff_low_s2,diff_low_s3,diff_low_s4,sma_diff_low,norm_sma_low,norm_wma_low,diff_log_vol,sma_diff_log_vol,diff_log_amount,sma_diff_log_amount
count,4.047079e+06,4.047079e+06,4.047079e+06,4.047079e+06,4.047079e+06,4.047079e+06,4.047079e+06,4.047079e+06,4.047079e+06,4.047079e+06,...,4.047079e+06,4.047079e+06,4.047079e+06,4.047079e+06,4.047079e+06,4.047079e+06,4.047079e+06,4.047079e+06,4.047079e+06,4.047079e+06
mean,1.994164e-03,1.774752e-03,1.586605e-03,2.441761e-02,3.291832e-02,3.220003e-02,2.399856e-17,2.287668e-17,3.369833e-17,3.827410e-18,...,1.455952e-17,-1.325548e-17,1.598778e-18,2.853879e-18,-1.151208e-17,-1.385242e-18,1.400242e-17,-1.173593e-17,2.066055e-17,1.216783e-17
std,2.848042e-01,1.329774e-01,1.449582e-01,2.230400e+00,1.039218e+00,1.129101e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-6.350000e-01,-2.930000e-01,-3.206667e-01,-4.490000e+00,-2.046661e+00,-2.244333e+00,-3.313361e+00,-3.312844e+00,-3.313065e+00,-3.312690e+00,...,-3.280353e+00,-3.280249e+00,-3.280050e+00,-3.162946e+00,-3.176289e+00,-3.206833e+00,-2.716579e+00,-2.751629e+00,-2.717637e+00,-2.759546e+00
25%,-1.200000e-01,-5.800000e-02,-6.333333e-02,-1.333300e+00,-6.300000e-01,-6.713333e-01,-4.972795e-01,-4.975681e-01,-4.975074e-01,-4.976044e-01,...,-4.407705e-01,-4.407205e-01,-4.409146e-01,-5.163607e-01,-4.836520e-01,-4.657443e-01,-6.752563e-01,-6.611769e-01,-6.759028e-01,-6.610347e-01
50%,0.000000e+00,2.284284e-15,6.938894e-18,0.000000e+00,3.000000e-02,2.534000e-02,-1.250608e-02,-1.230546e-02,-1.234787e-02,-1.223466e-02,...,3.810602e-02,3.814754e-02,3.818125e-02,2.044898e-02,2.563847e-02,3.256079e-02,-9.293069e-02,-5.989080e-02,-9.549179e-02,-6.374113e-02
75%,1.200000e-01,6.200000e-02,6.600000e-02,1.355700e+00,6.940000e-01,7.413333e-01,4.835162e-01,4.834947e-01,4.835118e-01,4.836749e-01,...,4.729201e-01,4.729028e-01,4.730134e-01,5.307077e-01,5.273723e-01,5.212694e-01,5.812313e-01,5.980222e-01,5.791432e-01,5.947486e-01
max,6.350000e-01,2.930000e-01,3.206667e-01,4.490000e+00,2.046661e+00,2.244333e+00,3.288349e+00,3.288233e+00,3.288369e+00,3.288221e+00,...,3.268422e+00,3.268134e+00,3.267953e+00,3.116776e+00,3.173908e+00,3.213383e+00,2.703455e+00,2.618152e+00,2.705767e+00,2.631336e+00


In [14]:
added_col=[
    "ctc_s0","ctc_s1","ctc_s2","ctc_s3","ctc_s4","wma_ctc","sma_ctc",
    "ts_code","trade_date","return",
]
df_daily_feature[added_col]=df_daily_clean[added_col]

## FE: gplearn 

In [21]:
from gplearn.genetic import SymbolicRegressor, SymbolicTransformer
from scipy.stats import pearsonr

In [378]:
df_daily_feature_train=df_daily_feature[df_daily_feature["trade_date"]<20200000]
df_daily_feature_test=df_daily_feature[df_daily_feature["trade_date"]>20200000]

df_daily_return_train=df_daily_return[df_daily_return["trade_date"]<20200000]
df_daily_return_test=df_daily_return[df_daily_return["trade_date"]>20200000]

X_train=np.array(df_daily_feature_train.iloc[:,:-2])
y_train=np.array(df_daily_return_train.loc[:,"return"])

X_test=np.array(df_daily_feature_test.iloc[:,:-2])
y_test=np.array(df_daily_return_test.loc[:,"return"])

In [52]:
function_set = ['add', 'sub', 'mul', 'div', 'log', 'sqrt', 'abs', 'neg', 'max', 'min']

gp = SymbolicTransformer(generations=3, population_size=1000,
                         hall_of_fame=100, n_components=10,
                         function_set=function_set,
                         parsimony_coefficient=0.0005,
                         max_samples=0.9, verbose=1,
                         random_state=0, n_jobs=10)

gp.fit(X_train,y_train)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    11.96        0.0118126        7        0.0518513        0.0470059     40.35m
   1     4.28        0.0280321        7        0.0518283        0.0472717     27.03m
   2     3.48        0.0345581        6        0.0618487        0.0639176     24.35m
   3     5.92        0.0297654        6        0.0622575        0.0602434     24.66m
   4     6.64        0.0316171        7        0.0642224        0.0641897     23.80m
   5     6.88        0.0430464       17        0.0714363        0.0728204     22.96m
   6     8.07        0.0421087       19        0.0743223        0.0707659     22.56m
   7     9.83        0.0420344       21        0.0771666        0.0753203     21.92m
   8    13.39        0.0465328       22         0.087711        0.0883511  

SymbolicTransformer(function_set=['add', 'sub', 'mul', 'div', 'log', 'sqrt',
                                  'abs', 'neg', 'max', 'min'],
                    generations=25, max_samples=0.9, n_components=3, n_jobs=80,
                    parsimony_coefficient=0.0005, random_state=0, verbose=1)

In [47]:
print(gp)
X_new=gp.transform(X_train)
for i in range(X_new.shape[-1]):
    print(f"{i} {pearsonr(X_new[:,i],y_train)[0]}")

[mul(mul(X36, X18), add(X32, X47)),
 sqrt(neg(sub(X21, X18))),
 sub(add(max(sqrt(sub(X21, X14)), log(abs(X1))), add(neg(min(X43, X4)), sub(log(X15), neg(X29)))), neg(abs(add(sub(X13, X22), add(-0.356, X6))))),
 log(min(X30, X10)),
 abs(X7),
 mul(X14, X46),
 sqrt(add(neg(X28), add(X16, X10))),
 mul(X8, X46),
 neg(mul(min(sub(sub(X43, X48), min(X35, X41)), sub(sub(X13, X8), mul(X10, X7))), min(div(log(X3), log(X45)), abs(max(X38, X44))))),
 max(X7, X40)]
0 -0.051369363595478
1 0.04368087215610385
2 0.03857874088585454
3 0.03783958612024296
4 0.03444672759015915
5 -0.032523298094035015
6 0.03258299984355576
7 0.031448304368178547
8 -0.02886343491306072
9 0.02796623738151545


In [65]:
df_f=pd.DataFrame(gp0.transform(X)[:,:3])
df_f["return"]=df_daily_feature["return"]
df_f.corr()

,0,1,2,3
0,1.000000,-0.024023,-0.102959,-0.546287
1,-0.024023,1.000000,0.629780,0.422797
2,-0.102959,0.629780,1.000000,0.598505
3,-0.546287,0.422797,0.598505,1.000000
